In [1]:
import socket
import threading
import time


In [4]:
def tcp_server(host='127.0.0.1', port=5001):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    print(f"[SERVER] Listening on {host}:{port}")
    
    conn, addr = server_socket.accept()
    print(f"[SERVER] Connected by {addr}")
    
    start = time.time()
    data = conn.recv(1024)  # Receive data
    end = time.time()
    
    print(f"[SERVER] Received data: {data.decode()}")
    print(f"[SERVER] Transmission time: {end-start:.6f} seconds")
    
    conn.close()
    server_socket.close()


In [5]:
def tcp_client(message="Hello Quantum!", host='127.0.0.1', port=5001):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    start = time.time()
    client_socket.send(message.encode())
    end = time.time()
    
    print(f"[CLIENT] Sent data: {message}")
    print(f"[CLIENT] Transmission time: {end-start:.6f} seconds")
    
    client_socket.close()


In [6]:
# Start server in a thread
server_thread = threading.Thread(target=tcp_server)
server_thread.start()

# Wait a moment for server to start
time.sleep(1)

# Run client
tcp_client("This is a test message for socket simulation!")

# Wait for server thread to finish
server_thread.join()


[SERVER] Listening on 127.0.0.1:5001
[SERVER] Connected by ('127.0.0.1', 64657)[CLIENT] Sent data: This is a test message for socket simulation!
[CLIENT] Transmission time: 0.000447 seconds

[SERVER] Received data: This is a test message for socket simulation!
[SERVER] Transmission time: 0.001209 seconds


In [7]:
import socket
import threading
import time
import os

FILE_PATH = r"C:\wireshark\synthetic_data_1mb.bin"
HOST = '127.0.0.1'
PORT = 5002
BUFFER_SIZE = 4096

# --- SERVER CODE ---
def tcp_server_file():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((HOST, PORT))
    server_socket.listen(1)
    print(f"[SERVER] Listening on {HOST}:{PORT}")

    conn, addr = server_socket.accept()
    print(f"[SERVER] Connected by {addr}")

    start = time.time()
    data_received = 0
    with open("received_file.bin", "wb") as f:
        while True:
            data = conn.recv(BUFFER_SIZE)
            if not data:
                break
            f.write(data)
            data_received += len(data)
    end = time.time()

    print(f"[SERVER] Received {data_received / 1024:.2f} KB")
    print(f"[SERVER] Transmission time: {end - start:.6f} seconds")

    conn.close()
    server_socket.close()


# --- CLIENT CODE ---
def tcp_client_file():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((HOST, PORT))

    file_size = os.path.getsize(FILE_PATH)
    print(f"[CLIENT] Sending file ({file_size / 1024:.2f} KB)...")

    start = time.time()
    with open(FILE_PATH, "rb") as f:
        while True:
            bytes_read = f.read(BUFFER_SIZE)
            if not bytes_read:
                break
            client_socket.sendall(bytes_read)
    end = time.time()

    print(f"[CLIENT] Transmission time: {end - start:.6f} seconds")

    client_socket.close()


# --- RUN BOTH TOGETHER ---
server_thread = threading.Thread(target=tcp_server_file)
server_thread.start()

# Give server a moment to start
time.sleep(1)

# Run client
tcp_client_file()

# Wait for server to finish
server_thread.join()


[SERVER] Listening on 127.0.0.1:5002
[SERVER] Connected by ('127.0.0.1', 60112)[CLIENT] Sending file (1024.00 KB)...

[CLIENT] Transmission time: 0.241183 seconds
[SERVER] Received 1024.00 KB
[SERVER] Transmission time: 0.255052 seconds
